In [13]:
# The following file is being used to test the Pipeline
import matplotlib.pyplot as plt
''' Import statements for all the files involved'''
from main import *
from utils import *
from wfield_utils import *
from registration import *
from hemocorrection import *
from denoising import *
from debug_visualize import *
from data_loading_functions import *
from data_preprocessor import *
from tqdm import tqdm
import pickle
import scipy.io

In [25]:
# The folllowing code is from main(). 
# params are global variables used to modify the code
params = {
    'path_to_session_data':       r"C:\Users\evilm\Downloads\07-29-2024",
    'pipe_num':           0,              # select desired pipeline number
    'get_raw_brightness': False,
    'get_DeltaF':         False,
    'get_zScore':         False
          }
# This is inside def main()
# For purposes of testing to visualise the data we will do analysis here.
session_file_paths = get_file_paths(params['path_to_session_data'])
#print(session_file_paths)
# create an obj for each recording within the session
recording = rawDataPreprocessor(params['path_to_session_data'])

print(f'{datetime.datetime.now().time()}: Done loading session data into object')

# Currently 
# recording.all_frames.shape
blue_frames = recording.all_frames[:,0,...]
analog_data = load_dat_analog(recording.path_to_analog)
frame_times = load_mat_frameTimes(recording.path_to_frameTimes)
blue_ftimes = frame_times['frameTimes']

olfac_matfile = r"C:\Users\evilm\Downloads\07-29-2024\data_mixed_full_grating_OdorStimulus_202407291652168_mat.mat"
olfac_configs = pickle.loads(scipy.io.loadmat(olfac_matfile)['configs'].tostring())
olfac_experiment_start_timestamp = pickle.loads(scipy.io.loadmat(olfac_matfile)['experiment_start_timestamp'].tostring())
#olfac_parameters = pickle.loads(scipy.io.loadmat(olfac_matfile)['parameters'].tostring())
# Skipping software environement
olfac_stim_frame_info = pickle.loads(scipy.io.loadmat(olfac_matfile)['stimulus_frame_info'].tostring())
#olfac_stim_frame_info_text = pickle.loads(scipy.io.loadmat(olfac_matfile)['stimulus_frame_info_text'].tostring())
olfac_user_data = pickle.loads(scipy.io.loadmat(olfac_matfile)['user_data'].tostring())
olfac_sync = scipy.io.loadmat(olfac_matfile)['sync']

14:30:12.794577: Done loading session data into object


C:\Users\evilm\AppData\Local\Temp\ipykernel_101948\934647974.py:27: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  olfac_configs = pickle.loads(scipy.io.loadmat(olfac_matfile)['configs'].tostring())
C:\Users\evilm\AppData\Local\Temp\ipykernel_101948\934647974.py:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  olfac_experiment_start_timestamp = pickle.loads(scipy.io.loadmat(olfac_matfile)['experiment_start_timestamp'].tostring())
C:\Users\evilm\AppData\Local\Temp\ipykernel_101948\934647974.py:29: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  olfac_parameters = pickle.loads(scipy.io.loadmat(olfac_matfile)['parameters'].tostring())
C:\Users\evilm\AppData\Local\Temp\ipykernel_101948\934647974.py:31: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  olfac_stim_frame_info = pickle.loads(scipy.io.loadmat(olfac_matfile)['stimulus_frame_info'].tostring())
C:\Users\evilm\AppData\Local\Temp\ipykernel_

In [57]:
#len(list(olfac_user_data)[0])
#  - list(olfac_stim_frame_info)[1::2]['time']
# Olfac Times
olfac_odortimes = np.array([time['time'] for time in list(olfac_stim_frame_info) ])
np.unique(olfac_odortimes[::2]-olfac_odortimes[1::2])

array([-6.01725245, -6.01708055, -6.01683092, -6.01664019, -6.01618314,
       -6.01606846, -6.01604295, -6.01603913, -6.0159061 , -6.01567531,
       -6.0155549 , -6.01553082, -6.0155201 , -6.01535964, -6.01529241,
       -6.01525378, -6.01517439, -6.01513648, -6.01498866, -6.01479578,
       -6.01469254, -6.01460528, -6.0145483 , -6.0137763 , -6.00616455])

In [75]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 100000

%matplotlib inline
plt.plot(olfac_sync[:,5])
plt.show()

IndexError: index 5 is out of bounds for axis 1 with size 5

In [ ]:
# Testing motion correction
#print(f'{datetime.datetime.now().time()}: Starting Motion Correction')
#_, _, motion_corrected_frames_all = motion_correct(dat=recordings['recording_1'].all_frames, out=None, mode='ecc', apply_shifts=True)
#print(f'{datetime.datetime.now().time()}: Done Motion Correction')
#np.save(params['path_to_session_data']+"\motion_corrected_data",motion_corrected_frames_all)

In [ ]:
motion_corrected_frames_all = np.load(r"C:\Users\evilm\Downloads\07-18-2024\motion_corrected_data.npy")

In [ ]:
blue_frames = motion_corrected_frames_all[0:2000,0,...]
violet_frames = motion_corrected_frames_all[0:2000,1,...]
motion_corrected_frames_all = []

# Testing Normalising 
blue_normalised_frames = [(frame-np.min(frame))/(np.max(frame)-np.min(frame)) for frame in tqdm(blue_frames[:])]
viol_normalised_frames = [(frame-np.min(frame))/(np.max(frame)-np.min(frame)) for frame in tqdm(violet_frames[:])]

hemo_corrected_frames = [(blue_normalised_frames[i] - viol_normalised_frames[i]) for i in tqdm(range(len(blue_normalised_frames)))]

In [ ]:
# %matplotlib inline
# import matplotlib.animation as animation
# plt.figure
# plt.imshow(blue_normalised_frames[0],cmap='gray')
# plt.show()

# plt.figure
# plt.imshow(viol_normalised_frames[0],cmap='gray')
# plt.show()

# hemo_corr = blue_normalised_frames[0]-viol_normalised_frames[0]
# plt.figure
# plt.imshow(hemo_corr,cmap='gray')
# plt.show()

# print(np.max(hemo_corr))
# print(np.min(hemo_corr))
# hemo_corr = (hemo_corr - np.min(hemo_corr))/(np.max(hemo_corr)-np.min(hemo_corr))

# plt.figure
# plt.imshow(hemo_corr,cmap='gray')
# plt.show()
# print(np.max(hemo_corr))
# print(np.min(hemo_corr))

frames = []
fig = plt.figure()
for i in tqdm(range(len(hemo_corrected_frames))):
    frames.append([plt.imshow(hemo_corrected_frames[i],
                             cmap=plt.cm.Greys_r, animated = True)])
ani = animation.ArtistAnimation(fig,frames,interval=40, blit = True, repeat_delay=1000)
#ni.save('movie.mp4')
plt.show()


In [ ]:
# Testing Denoising and Compression
motion_corrected_frames_all = np.load(r"C:\Users\evilm\Downloads\07-18-2024\motion_corrected_data.npy")
# 2. Denoising and Compression
print(f'{datetime.datetime.now().time()}: Starting  Blue Frames')
denoised_blue_frames, blueU, blueS, blueVT = denoise_svd(motion_corrected_frames_all[:,0,...],rank=200) # only denoise blue channel for now
print(f'{datetime.datetime.now().time()}: Done Denoising Blue Frames')

print(f'{datetime.datetime.now().time()}: Starting  Violet Frames')
denoised_violet_frames, violetU, violetS, violetVT = denoise_svd(motion_corrected_frames_all[:,1,...],rank=200) # only denoise blue channel for now
print(f'{datetime.datetime.now().time()}: Done Denoising Violet Frames')

np.save(params['path_to_session_data']+"\denoised_blue_frames",denoised_blue_frames)
np.save(params['path_to_session_data']+"\denoised_violet_frames",denoised_violet_frames)
np.save(params['path_to_session_data']+"\blueU",blueU)
np.save(params['path_to_session_data']+"\blueS",blueS)
np.save(params['path_to_session_data']+"\blueVT",blueVT)
np.save(params['path_to_session_data']+"\violetU",violetU)
np.save(params['path_to_session_data']+"\violetS",violetS)
np.save(params['path_to_session_data']+"\violetVT",violetVT)

In [ ]:
#get_recording_paths(params['path_to_session_data'])
'''
data = recordings['recording_1'].demo_pipeline()
data = np.array(data)
np.save(params['path_to_session_data']+"\blue_frame_data",data)
'''

In [ ]:
#data = np.load(r"C:\Users\evilm\Downloads\07-18-2024\blue_frame_data.npy")

In [ ]:
#%matplotlib inline
#plt.imshow(np.mean(data,axis=0),cmap="gray")
#plt.show()